## Подключение к API dota

### Импорт библиотек и создание функций

In [1]:
#импорт библиотеки requests
import requests

#импорт библиотеки json
import json

#импорт библиотеки pprint
import pprint

#импорт pandas
import pandas as pd

import time, os
from pathlib import Path

In [2]:
# Задаем переменную c url API 
API = "https://api.opendota.com/api"

In [6]:
# Создадим функцию для отправки запросов get
def get_json(url, params=None):
    """функция get_json:
    - принимает url
    - отправляет запрос get с использованием библиотеки requests
    - Возвращает от API json файл 
    - преобразует json в объект языка Python (словарь или список)"""
    r = requests.get(
        url, 
        params=params or {}, 
        timeout=30, 
        headers={"User-Agent": "opendota-ds"})
    r.raise_for_status()
    return r.json()

In [7]:
# Создадим функции для чтения и загрузки данных, а также сохранения памяти прогресса
# функции нужны для того,чтобы возобновлять процесс там, где остановлись и не запрашивать лишнее у API
def load_json_list(path):
    """
    Функция проверяет, существует ли файл 
    Если да: открывает JSON и возвращает список матчей
    Если нет: возвращает пустой список
    """
    if Path(path).exists():
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    return []

def save_json_list(path, data_list):
    """" Открывает файл на запись и сохраняет туда список (match_ids)"""
    with open(path, "w", encoding="utf-8") as f:
        json.dump(list(data_list), f, ensure_ascii=False, indent=2)

### Загрузка id открытых матчей

In [8]:
# зарузка данных с использованием метода  GET /publicMatches и учетом пагинации (1 стр - 100 матчей)
# сколько страниц хотим загрузить
pages_public = 1
# файл, где хранится полный список собранных match_id
match_ids_file = "match_ids.json"

# загружаем ранее собранные ID из файла, если он есть, match_ids подхватывает прошлый прогресс
# это упорядоченный спискок всех найденных ID
match_ids = load_json_list(match_ids_file)

# определяем множество для быстрой проверки дублей
seen = set(match_ids)

# курсор пагинации, на первой итерации равен None
less = None

# проходимся по странице, передаем id матчей. Основной цикл: листаем страницы и собираем ID
# less_than_match_id - курсор для перемещения по страницам при пагинации, на первой итерации запрос без параметров, на следующих - просим более старые
# batch - одна страница результатов, список из 100 матчей
for _ in range(pages_public):
    params = {"less_than_match_id": less} if less else None
    batch = get_json(f"{API}/publicMatches", params=params)
    if not batch:
        break
# достаем из ответа только match_id из каждого словаря страницы
    ids = [m["match_id"] for m in batch if "match_id" in m]
    # проверяем mid в seen, новый добавляем в seen и в match_ids
    for mid in ids:
        if mid not in seen:
            seen.add(mid)
            match_ids.append(mid)
# обновляем курсор, берем минимальный матч на этой страницы
    less = min(ids) if ids else less
    time.sleep(1.2)  # условие, чтобы не превысить лимит в 60 запросов на бесплатном доступе

# сохраняем обновленный список
save_json_list(match_ids_file, match_ids)
# печатаем итоговое количество
print(f"Всего скачано матчей: {len(match_ids)}")


Всего скачано матчей: 20386


### Скачивание файлов с информацией о матчах

In [17]:
# скачаем полные карточки матчей для подмножества match_ids и сохраним сырые матчи в файл JSONL
# цель: не превышать лимиты бесплатного API
# не скачивать 1 матч повторно
# не терять прогресс, если сессия обрывается

# Добавим количество матчей, которые мы ходим скачать за запуск скрипта (200-500 шт)
match_count = 100 
# Присвоим переменной downloaded_file название файла со списком match_id, которые уже скачаны
downloaded_file = "downloaded_ids.json" 

# Присвоим переменной jsonl_path название файла с основным хранилищем сырых матчей в формате JSONL (каждый матч — отдельная строка JSON)
jsonl_path = "matches_raw.jsonl" 

# загружаем уже скачанные match_id с использованием функции load_json_list, превращаем во множество
downloaded_ids = set(load_json_list(downloaded_file))

# берём только те match_id из списка match_ids, которые еще не скачаны, ограничившись первыми HYDRATE_N
to_fetch = [m_id for m_id in match_ids if m_id not in downloaded_ids][:match_count]
print(f"Добавляем {len(to_fetch)} матчей...")

# Проверяем, что существует JSONL-файл на диске (по одной строке на матч), в который пишем матчи построчно
Path(jsonl_path).touch(exist_ok=True)


# скачиваем /matches/{id} и сразу пишем на диск
with open(jsonl_path, "a", encoding="utf-8") as f:
    for m_id in to_fetch: 
        try:
            url = f"{API}/matches/{m_id}"
            try:
                data = get_json(url)
                

                
            except requests.HTTPError as e:
                # При ошибке 429 (Too Many Requests) повторяем
                if e.response is not None and e.response.status_code == 429:
                    time.sleep(2.0)
                    data= get_json(url)  
                else:
                    raise
           # записываем сразу построчно информацию о матчах, если что-то упадёт, прогресс не будет утерян
            f.write(json.dumps(data, ensure_ascii=False) + "\n")
            downloaded_ids.add(m_id)
        except Exception:
            raise
        time.sleep(1.2)           
# сохраняем актуальный список уже скачанных ID
save_json_list(downloaded_file, list(downloaded_ids))
print(f"Всего скачано: {len(downloaded_ids)}")

Добавляем 100 матчей...
Всего скачано: 19101


In [18]:
# готовим хранилище для raw-данных
raw_matches = []

# Загружаем матчи из JSONL
with open("matches_raw.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            raw_matches.append(json.loads(line))
print(f"Загружено {len(raw_matches)} матчей из JSONL")

Загружено 20107 матчей из JSONL


### Создание датафреймов c информацией о матчах и об игроках и выгрузка из в CSV файлы

#### Создание датафрейма df_matches с информацией о матчах

In [19]:
# Создаем датафрейм df_matches

df_matches_new = pd.DataFrame([{
    "match_id": d.get("match_id"),
    "duration": d.get("duration"),
    "first_blood_time": d.get("first_blood_time"),
    "radiant_win": d.get("radiant_win"),
    "region": d.get("region"),
} for d in raw_matches if isinstance(d, dict)])

In [20]:
# загрузим ранее скачанные матчи, если они существуют
matches_path = "df_matches.csv"
if os.path.exists(matches_path):
    df_matches_old = pd.read_csv(matches_path)
    df_matches = pd.concat([df_matches_old, df_matches_new], ignore_index=True)
    df_matches.drop_duplicates(subset=["match_id"], inplace=True)
else:
    df_matches = df_matches_new

In [21]:
# сохраняем csv
df_matches.to_csv(matches_path, index=False)
print(f"Saved {len(df_matches)} rows to df_matches.csv")

Saved 19358 rows to df_matches.csv


#### Создание датафрейма df_players с информацией об игроках

In [22]:
# Создаем датафрейм df_players
df_players_new = pd.DataFrame([
    {
        "match_id": d.get("match_id"),
        "player_slot": p.get("player_slot"),
        "steam_id": p.get("account_id"),
        "personaname": p.get("personaname"),
        "assists": p.get("assists"),
        "deaths": p.get("deaths"),
        "denies": p.get("denies"),
        "gold_per_min": p.get("gold_per_min"),
        "hero_id": p.get("hero_id"),
        "item_0": p.get("item_0"),
        "item_1": p.get("item_1"),
        "item_2": p.get("item_2"),
        "item_3": p.get("item_3"),
        "item_4": p.get("item_4"),
        "item_5": p.get("item_5"),
        "kills": p.get("kills"),
        "xp_per_min": p.get("xp_per_min"),
        "is_radiant": p.get("isRadiant"),
        "win": p.get("win"),
        "last_hits": p.get("last_hits"),
        "hero_damage": p.get("hero_damage"),
        "hero_healing": p.get("hero_healing"),
        "tower_damage": p.get("tower_damage"),
    }
    for d in raw_matches if isinstance(d, dict)
    for p in d.get("players", [])
])


In [23]:
# загружаем файл, если он существует или создаем новый
players_path = "df_players.csv"
if os.path.exists(players_path):
    df_players_old = pd.read_csv(players_path)
    df_players = pd.concat([df_players_old, df_players_new], ignore_index=True)
    df_players.drop_duplicates(subset=["match_id", "player_slot"], inplace=True)
else:
    df_players = df_players_new

In [24]:
# сохраняем csv
df_players.to_csv(players_path, index=False)
print(f"Сохранено {len(df_players)} строк в df_players.csv")

Сохранено 192580 строк в df_players.csv


## Загрузка информации о героях с сайта Дота

In [25]:
batch_heroes = get_json(f"{API}/heroStats")

In [26]:
df_heroes = pd.DataFrame([{
    "id": h.get("id"),
    "localized_name": h.get("localized_name"),
    "attack_type": h.get("attack_type"),
    "primary_attr": h.get("primary_attr"),
    "roles": h.get("roles"),
    "base_health": h.get("base_health"),
    "base_health_regen": h.get("base_health_regen"),
    "base_mana": h.get("base_mana"),
    "base_mana_regen": h.get("base_mana_regen"),
    "base_armor": h.get("base_armor"),
    "base_mr": h.get("base_mr"),
    "base_attack_min": h.get("base_attack_min"),
    "base_attack_max": h.get("base_attack_max"),
    "base_str": h.get("base_str"),
    "base_agi": h.get("base_agi"),
    "base_int": h.get("base_int"),
    "str_gain": h.get("str_gain"),
    "agi_gain": h.get("agi_gain"),
    "int_gain": h.get("int_gain"),
    "attack_range": h.get("attack_range"),
    "projectile_speed": h.get("projectile_speed"),
    "attack_rate": h.get("attack_rate"),
    "base_attack_time": h.get("base_attack_time"),
    "attack_point": h.get("attack_point"),
    "move_speed": h.get("move_speed"),
    "turn_rate": h.get("turn_rate"),
    "cm_enabled": h.get("cm_enabled"),
    "legs": h.get("legs"),
    "day_vision": h.get("day_vision"),
    "night_vision": h.get("night_vision"),
} for h in batch_heroes])

In [27]:
df_heroes.to_csv("df_heroes.csv", index=False, encoding="utf-8")